# SCRAPING

### GET LINKS

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import json

# Configuration du WebDriver
chrome_driver_path = "/usr/local/bin/chromedriver"
chrome_options = Options()
chrome_options.add_argument("--headless")  # Mode sans interface graphique
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Ouvrir la page Senego
driver.get("https://senego.com/rubrique/actualites")
time.sleep(3)  # Attendre le chargement initial

# Fonction pour faire défiler la page
def scroll_down():
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(2)  # Attendre le chargement

# Récupérer les liens des articles à partir des balises <article>
article_links = set()
while len(article_links) < 400:
    scroll_down()
    posts_section = driver.find_element(By.CSS_SELECTOR, "section.postsSectionCenter")
    articles = posts_section.find_elements(By.CSS_SELECTOR, "article a[href]")
    for article in articles:
        # print("article: ", article)
        try:
            link = article.get_attribute("href")
            # print("link:",link)
            article_links.add(link)
        except Exception as e:
            print(f"Erreur sur {link} : {e}")
            continue


# Limiter à 300 liens uniques
article_links = list(article_links)[:400]
print(f"Articles collectés : {len(article_links)}")


Articles collectés : 400


### DATA EXTRACTION

In [4]:
# Extraction des données des articles
articles_data = []
id = 0
# i=0
for  link in article_links[:200]:
    print(id)
    try:
        driver.get(link)
    except Exception as e:
        print("impossible", link)
    
    time.sleep(2)  # Attendre le chargement de la page

    try:
        title = driver.find_element(By.TAG_NAME, "h1").text
        # print("title", id+1, ":", title)
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "p")
        content = "\n".join([p.text for p in paragraphs])
        time_element = driver.find_element(By.TAG_NAME, "time")
        date = time_element.get_attribute("datetime")  # Format ISO 8601
        # print(f"Date : {date}")
        articles_data.append({"id" : id, "title": title, "url": link, "content": content})
        id+=1
        # print(f"Article {index+1}/{len(article_links)} collecté")
    except Exception as e:
        print(f"Erreur sur {link} : {e}")
    
# # Fermer le navigateur
# driver.quit()

# # Sauvegarde des articles dans un fichier JSON
# with open("senego_articles.json", "w", encoding="utf-8") as f:
#     json.dump(articles_data, f, ensure_ascii=False, indent=4)

# print("Extraction terminée ! Articles sauvegardés dans 'senego_articles.json'.")
    



Erreur sur https://senego.com/gestion-de-dakarnave-letat-met-fin-a-des-annees-de-gabegie-et-reprend-le-controle_1803165.html : Message: tab crashed
  (Session info: chrome=133.0.6943.53)
Stacktrace:
#0 0x594d5054109a <unknown>
#1 0x594d50008703 <unknown>
#2 0x594d4ffef716 <unknown>
#3 0x594d4ffed479 <unknown>
#4 0x594d4ffedeaf <unknown>
#5 0x594d4fffd8ab <unknown>
#6 0x594d5001684f <unknown>
#7 0x594d5001cf7b <unknown>
#8 0x594d4ffee5c0 <unknown>
#9 0x594d50016336 <unknown>
#10 0x594d500a5a5c <unknown>
#11 0x594d5007f753 <unknown>
#12 0x594d5004b38e <unknown>
#13 0x594d5004cb51 <unknown>
#14 0x594d5050a76b <unknown>
#15 0x594d5050e6f2 <unknown>
#16 0x594d504f68fc <unknown>
#17 0x594d5050f2e4 <unknown>
#18 0x594d504da9cf <unknown>
#19 0x594d5052fcd8 <unknown>
#20 0x594d5052feb6 <unknown>
#21 0x594d5053ff16 <unknown>
#22 0x732d7bc9ca94 <unknown>
#23 0x732d7bd29c3c <unknown>

88
impossible https://senego.com/residence-du-president-de-lassemblee-habitee-par-la-2%e1%b5%89-epouse-du-presiden